In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# os.environ['KAGGLE_CONFIG_DIR'] = "/content"
# !kaggle competitions download -c tabular-playground-series-aug-2022
# !unzip tabular-playground-series-aug-2022
# for dirname, _, filenames in os.walk('/content'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 88% 2.00M/2.27M [00:00<00:00, 3.01MB/s]
100% 2.27M/2.27M [00:00<00:00, 2.92MB/s]
Archive:  tabular-playground-series-aug-2022.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
/content/kaggle.json
/content/0711257_model.pkl
/content/tabular-playground-series-aug-2022.zip
/content/sample_submission.csv
/content/train.csv
/content/test.csv
/content/.config/config_sentinel
/content/.config/.last_update_check.json
/content/.config/gce
/content/.config/.last_opt_in_prompt.yaml
/content/.config/active_config
/content/.config/.last_survey_prompt.yaml
/content/.config/logs/2023.01.05/14.32.48.311993.log
/content/.config/logs/2023.01.05/14.34.01.063009.log
/content/.config/logs/2023.01.05/14.34.00.192801.log
/content/.config/logs/2023.01.05/14.33.30.524625.log
/content/.config/logs/2023.01.05/14.32.18.784307.log
/content/.config/logs/2023.01.05/14.33.16.757082.log
/content/.config/configurations/config_default
/content/sample_

In [2]:
test = pd.read_csv('/content/test.csv')
train = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')
model_path = "/content/0711257_model.pkl"

In [3]:
!pip install feature_engine
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 KB 5.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.6 MB/s eta 0:00:00


In [4]:
import os
import sys
import joblib
import numpy as np
import pandas as pd
import pickle
import gc; gc.enable()
from lightgbm import LGBMClassifier
from sklearn.impute import KNNImputer
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB
from feature_engine.encoding import WoEEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression, HuberRegressor
import warnings; warnings.filterwarnings("ignore")
from tensorflow.keras import Sequential  
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_addons as tfa

pd.options.display.max_columns = 999

In [5]:
with open(model_path, 'rb') as fp:
    models = pickle.load(fp)

In [6]:
folds_dict = {'Fold 1': [['C', 'D', 'E'], ['A', 'B']], 
               'Fold 2': [['B', 'D', 'E'], ['A', 'C']],
               'Fold 3': [['B', 'C', 'E'], ['A', 'D']],
               'Fold 4': [['B', 'C', 'D'], ['A', 'E']],
               'Fold 5': [['A', 'D', 'E'], ['B', 'C']],
               'Fold 6': [['A', 'C', 'E'], ['B', 'D']],
               'Fold 7': [['A', 'C', 'D'], ['B', 'E']],
               'Fold 8': [['A', 'B', 'E'], ['C', 'D']],
               'Fold 9': [['A', 'B', 'D'], ['C', 'E']],
               'Fold 10': [['A', 'B', 'C'], ['D', 'E']]}
print(folds_dict["Fold 1"])

[['C', 'D', 'E'], ['A', 'B']]


In [7]:
tf.keras.utils.set_random_seed(42)
def preprocessing(df_train, df_test):
    data = pd.concat([df_train, df_test])
    features = 0
    
    data['m3_missing'] = data['measurement_3'].isnull().astype(np.int8)
    data['m4_missing'] = data['measurement_4'].isnull().astype(np.int8)
    data['m5_missing'] = data['measurement_5'].isnull().astype(np.int8)
    data['area'] = data['attribute_2'] * data['attribute_3']

    # dictionnary of dictionnaries (for the 11 best correlated measurement columns), 
    # we will use the dictionnaries below to select the best correlated columns according to the product code)
    # Only for 'measurement_17' we make a 'manual' selection :
    full_fill_dict ={}
    full_fill_dict['measurement_17'] = {
        'A': ['measurement_5','measurement_6','measurement_8'],
        'B': ['measurement_4','measurement_5','measurement_7'],
        'C': ['measurement_5','measurement_7','measurement_8','measurement_9'],
        'D': ['measurement_5','measurement_6','measurement_7','measurement_8'],
        'E': ['measurement_4','measurement_5','measurement_6','measurement_8'],
        'F': ['measurement_4','measurement_5','measurement_6','measurement_7'],
        'G': ['measurement_4','measurement_6','measurement_8','measurement_9'],
        'H': ['measurement_4','measurement_5','measurement_7','measurement_8','measurement_9'],
        'I': ['measurement_3','measurement_7','measurement_8']
    }

    # collect the name of the next 10 best measurement columns sorted by correlation (except 17 already done above):
    col = [col for col in df_test.columns if 'measurement' not in col]+ ['loading','m3_missing','m5_missing']
    # print(col)
    a = []
    b =[]

    #sort the measurement as its correlation among measurement_3-17
    corr = np.absolute(data.drop(col, axis=1).corr())
    for x in range(3,17):
        corrr = corr[f'measurement_{x}'].sort_values(ascending=False)
        a.append(np.round(np.sum(corrr[1:4]),3)) # we add the 3 first lines of the correlation values to get the "most correlated"
        b.append(f'measurement_{x}')
    print(a,b)
    c = pd.DataFrame()
    c['Selected columns'] = b
    c['correlation total'] = a
    c = c.sort_values(by = 'correlation total',ascending=False).reset_index(drop = True)
    print(f'Columns selected by correlation sum of the 3 first rows : ')
    display(c.head(15))

    for i in range(10): 
        measurement_col = c.iloc[i, 0]
        fill_dict = {}
        for x in data.product_code.unique() : 
            corr = np.absolute(data[data.product_code == x].drop(col, axis=1).corr()[measurement_col]).sort_values(ascending=False)
            measurement_col_dic = {}
            fill_dict[x] = corr[1:5].index.tolist()
        full_fill_dict[measurement_col] = fill_dict

    feature = [f for f in data.columns if f.startswith('measurement') or f == 'loading']
    print('failure' in feature)
    nullValue_cols = [col for col in df_train.columns if df_train[col].isnull().sum()]
    for code in data.product_code.unique():
        total_na_filled_by_linear_model = 0
        print(f'\n-------- Product code {code} ----------\n')
        print(f'filled by linear model :')
        for measurement_col in list(full_fill_dict.keys()):
            tmp = data[data.product_code == code]
            column = full_fill_dict[measurement_col][code]
            tmp_train = tmp[column+[measurement_col]].dropna(how='any') # a na in row then drop
            '''
            every column should have value && measurement should not
            '''
            tmp_test = tmp[(tmp[column].isnull().sum(axis=1)==0)&(tmp[measurement_col].isnull())]
            model = HuberRegressor(epsilon=1.9)
            model.fit(tmp_train[column], tmp_train[measurement_col])
            data.loc[(data.product_code==code)&(data[column].isnull().sum(axis=1)==0)&(data[measurement_col].isnull()),measurement_col] = model.predict(tmp_test[column])
            print(f'{measurement_col} : {len(tmp_test)}')
            total_na_filled_by_linear_model += len(tmp_test)
            # break
        # others NA columns:
        # print(data.loc[data["product_code"] == code, nullValue_cols].isnull().sum())
        # print(data.loc[data["product_code"] == code, nullValue_cols].isnull().sum().sum())
        NA = data.loc[data["product_code"] == code, nullValue_cols].isnull().sum().sum()
        '''
        KNNImputer
        '''
        model1 = KNNImputer(n_neighbors=3)
        data.loc[data.product_code==code, feature] = model1.fit_transform(data.loc[data.product_code==code, feature])
        print(f'\n{total_na_filled_by_linear_model} filled by linear model ') 
        print(f'{NA} filled by KNN ')
        # break
    data['measurement_avg'] = data[[f'measurement_{i}' for i in range(3, 17)]].mean(axis=1)
    data['measurement_std'] = data[[f'measurement_{i}' for i in range(3, 17)]].std(axis=1)
    data['measurement_median'] = data[[f'measurement_{i}' for i in range(3, 17)]].median(axis=1)
    data['measurement_max'] = data[[f'measurement_{i}' for i in range(3, 17)]].max(axis=1)
    data['measurement_min'] = data[[f'measurement_{i}' for i in range(3, 17)]].min(axis=1)
    data['measurement_skew'] = data[[f'measurement_{i}' for i in range(3, 17)]].skew(axis=1)
    
    
    df_train = data.iloc[:df_train.shape[0],:]
    df_test = data.iloc[df_train.shape[0]:,:]

    '''
    drop attribute_1 cuz they have different unique values between train and test
    '''
    woe_encoder = WoEEncoder(variables=['attribute_0'])
    woe_encoder.fit(df_train, df_train['failure'])
    df_train = woe_encoder.transform(df_train)
    df_test = woe_encoder.transform(df_test)
    return df_train, df_test
df_train, df_test = preprocessing(train, test)

[0.092, 0.331, 0.386, 0.365, 0.336, 0.454, 0.201, 0.3, 0.395, 0.145, 0.188, 0.225, 0.301, 0.252] ['measurement_3', 'measurement_4', 'measurement_5', 'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9', 'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13', 'measurement_14', 'measurement_15', 'measurement_16']
Columns selected by correlation sum of the 3 first rows : 


,Selected columns,correlation total
0,measurement_8,0.454
1,measurement_11,0.395
2,measurement_5,0.386
3,measurement_6,0.365
4,measurement_7,0.336
5,measurement_4,0.331
6,measurement_15,0.301
7,measurement_10,0.300
8,measurement_16,0.252
9,measurement_14,0.225


False

-------- Product code A ----------

filled by linear model :
measurement_17 : 386
measurement_8 : 175
measurement_11 : 225
measurement_5 : 113
measurement_6 : 146
measurement_7 : 166
measurement_4 : 79
measurement_15 : 273
measurement_10 : 209
measurement_16 : 293
measurement_14 : 237

2302 filled by linear model 
1547 filled by KNN 

-------- Product code B ----------

filled by linear model :
measurement_17 : 418
measurement_8 : 165
measurement_11 : 220
measurement_5 : 90
measurement_6 : 106
measurement_7 : 176
measurement_4 : 80
measurement_15 : 294
measurement_10 : 197
measurement_16 : 358
measurement_14 : 330

2434 filled by linear model 
1541 filled by KNN 

-------- Product code C ----------

filled by linear model :
measurement_17 : 391
measurement_8 : 211
measurement_11 : 231
measurement_5 : 141
measurement_6 : 150
measurement_7 : 140
measurement_4 : 110
measurement_15 : 319
measurement_10 : 262
measurement_16 : 343
measurement_14 : 340

2638 filled by linear model 
170

In [8]:
features = ['loading', 'attribute_0', 'measurement_17',
            'measurement_0', 'measurement_1', 'measurement_2',
            'area', 'm3_missing',
            'm4_missing', 'm5_missing', 'measurement_avg',
            'measurement_std', 'measurement_median','measurement_min',
            'measurement_skew']
features

['loading',
 'attribute_0',
 'measurement_17',
 'measurement_0',
 'measurement_1',
 'measurement_2',
 'area',
 'm3_missing',
 'm4_missing',
 'm5_missing',
 'measurement_avg',
 'measurement_std',
 'measurement_median',
 'measurement_min',
 'measurement_skew']

In [9]:
pred = np.zeros((df_test.shape[0], 1))
weight = models[-1]
for i, fold in enumerate(folds_dict.keys()):
  model = models[i]
  pred += model.predict(df_test[features].values).reshape(-1, 1) * weight[i]
pred /= np.sum(weight)
submission['failure'] = pred
submission.to_csv('0711257_submission.csv', index=False)

650/650 [==============================] - 1s 2ms/step
